# Mapper 2 example notebook

The following notebook goes over the basics of using Mapper2 function and the workflow. Much of the functionality is the same as previously described in Mapper1 per Remco procedure. As such the documentation for Mapper 1 largely still applies to Mapper 2. Separate notebooks have been created for more in-depth discussion and showcase of the various functions and functionalities created for and within Mapper 2.

quick overview of mapper:
- Mapper 2, modification of viewser3 Remco procedure
- Allows for built in customization of the map scales
- Allows for addition of backgrounds
- Now comes equiped with transparencies and masking
- Simplifies label creation in case of recursive map writing
- Simplifies bounding box lookup
- cmap still specifies color, defaults to rainbow if no color is selected


Mapping uses geopandas. (https://geopandas.org/en/stable/). Essentially a combination of shapely and pandas. 
Shapely is a package that is able to present and manipulate spatial geometries, which are themselves based on GIS (geo information systems).

Mapper functions by essentially associating certain areas (e.g. country or priogrid cell) with pre-determined geometries. It then can use layers to map things on top of each to create a final image.

In [ ]:
#verify the version. 
#match it to the latest version of views_mapper2 as described in the read me portion of the message. 
!pip show views_mapper2

In [ ]:
#step 1 obtain your data
#obtain data, in this example I am pulling the data from viewser queryset
from viewser import Queryset, Column

queryset = (Queryset("Mapper_tester_cm", "country_month")
            # CONFLICT TYPES NOT LOGGED and NOT time lagged
            .with_column(Column("ged_best_sb", from_table="ged2_cm", from_column="ged_sb_best_sum_nokgi")
                         .transform.missing.fill()
                         )
            # .with_column(Column('name', from_table='country', from_column= 'name')
            # the above does not seem to want to work

            .with_theme("fatalities")

            .describe("""Fatalities conflict history, cm level
            Mapper testing

            """)

            )

cm_master = queryset.publish().fetch()

In [ ]:
#fetch the multipolygons for mapping
#this fetches the association between country_id and geometries from viewser
#of note the geometries are based on the latest version of Cshapes and priogrid in use by viewser
#the set to_crs is setting the coordinate reference system (various projections of the map)
#set to default 4326 for this project

import geopandas as gpd
import sqlalchemy as sa
from ingester3.config import source_db_path

engine = sa.create_engine(source_db_path)
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)

In [ ]:
#import functions for mapping
#you can go over the specific description of the BBoxWriter, dictionary_writer and label writer in separate notebooks
import os
from views_mapper2.mapper2 import *
from views_mapper2.BBoxWriter import *
from views_mapper2.dictionary_writer import *
from views_mapper2.label_writer import *

In [ ]:
#specify savefile in case of save procedure
#this will allow you to test the save function
home = os.path.expanduser("~")
folder = '/Desktop/delete_when_done'
make_a_folder(home+folder)


## Basics of mapper2

component 1: init layer (creates the first basic layer)
    - width: Integer value for width in inches.
    - height: Integer value for height in inches.
    - frame_on: Bool for whether to draw a frame around the map. (for standard presention set to true)
    - title: Optional default title at matplotlib's default size.
    - bbox: List for the bbox per [left, right, bottom, top].
    - figure: Optional tuple of (fig, size) to use if you want to plot into an
        already existing fig and ax, rather than making a new one. (for standard presentation set to None)
    - cmap: color map, can be chosen at a later stage, defaults to rainbow (for standard presentation leave this out)
component 2: add layer (creates the layer with our data)
    - gdf: Geopandas GeoDataFrame to plot.
    - cmap: Optional matplotlib colormap object or string reference(e.g. "viridis"). 
        Defaults to rainbow if not specified.
    - map_scale: set a manual scale for the map. If missing defaults to the Remco procedure 
        (Remco procedure = set the min and max values to whatever is min and max values in the data).
    - map_dictionary: set manual scale with labels for the map. If missing defaults to Remco procedure.
        note: mapper preferentially looks for map_dictionary first, then map_scale, and then Remco default
        note: no need to use ticklabel and tickvalue commands as the code is subsumed within Mapper2
        note: Refer to dictionary writer for easy code for preset dictionaries
    - transparency: on a scale from 0 to 1, transparency communicates with the sidebar
        note: transparency 0 is fully transparent, tranparency 1 is fully opaque
        note: in retrospect this should have been called opacity
    - background: sets the background for the geopandas, uses premade tiles from contextlily library
        note: for more information, look at separate notebook
    - views_experimental_labels: displays country names, some issue with label collision for multiple countries
        note: choose numeric value for font size
        note: recommended use for country specific mapping with small bbox size
    - edgecolor: the color of the lines around the geometry objects (either countries or priogrids)
    - linewidth: the line size of the geometry objects
    - column: which column to graph, your variable
component 3: add mask layer (look at separate notebook for description)
component 4: add colorbar (can be used to modify colorbar, but is not needed, please ignore or review Remco notebooks for further information)
component 5: add views textbox (creates a textbox with information on the map and VIEWS logo)
    - text: enter a string you want to be in the textbox
    - textsize: integer for text size
component 6: save (save the output, include file name with full path)

In [ ]:
#example of country month mapping
#note this run uses pre-set scale/labelling built into mapper2, refer to dictionary notebook for more info

data= cm_master.copy()
gdf = gdf_ci_master.copy()

data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

mapper_showcase_cm=Mapper2(
    width=20,
    height=20,
    frame_on=True,
    title='mask with layers',
    bbox=bbox_from_cid('ame'), 
    figure = None
).add_layer(
    gdf=gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'binary',
    transparency = 1.0,
    edgecolor="black",
    linewidth=0.5,
    column="ged_best_sb"
).add_mask(
    gdf = gdf.loc[450],
    map_dictionary = dictionary_stand,
    cmap = 'rainbow',
    transparency = 1,
    masking_location = 'Nigeria',
    views_experimental_labels = None,
    column='ged_best_sb',
    background = None,
    edgecolor="black",
    linewidth=1
).add_views_textbox(
    text='hello world, hello world',
    textsize=20)

mapper_showcase_cm.save(home+folder+'/mapper_showcase_cm')

In [ ]:
#a quick demonstration of mapper as described above and the difference scale vs. dictionary makes

#Please note that much like previous iteration, mapper2 builds layers
#scale versus dictionary

#create custom scale and dictionary that makes it easy to see changes
ludicrous_scale = [0,100,500]
ludicrous_dictionary = {'lemon':0, 'apple':100, 'pineapple': 500}


#this step prepares data for mapping
#note we are combining the data with the geometry dataframes to create a master gdf dataframe that is used below

data= cm_master.copy()
gdf = gdf_ci_master.copy()
data = data.join(gdf.set_index("country_id"))
gdf = gpd.GeoDataFrame(data, geometry="geom")

show_me_scale=Mapper2(
    width=10,   #dimension width
    height=10,  #dimension height
    frame_on=True,
    title="scale",    #title
    bbox=bbox_from_cid('ame')    #bounding box
).add_layer(
    gdf=gdf.loc[457],  #what do you want to map, month_id 457 in this case
    map_scale = ludicrous_scale,  #choice of scale
    cmap='rainbow',    #choice of cmap
    edgecolor="black",   #border color choice
    linewidth=0.5,    #line size choice
    column="ged_best_sb"   #which column do you want to graph
)

show_me_dictionary = Mapper2(
    width=10,
    height=10,
    frame_on=True,
    title="dictionary",
    bbox=bbox_from_cid('ame'),
).add_layer(
    gdf=gdf.loc[457],
    map_dictionary = ludicrous_dictionary,
    cmap='rainbow',
    edgecolor="black",
    linewidth=0.5,
    column="ged_best_sb"
)

#below runs the save function, comment out if you do not wish to save

show_me_scale.save(home+folder+'/show_me_scale')
show_me_dictionary.save(home+folder+'/show_me_dictionary')

## Functionality with PG level
- Similar to the above and previous Remco procedure
- note that since geometries are squares, an additional step (referred to as Jim procedure) is required to overlay country borders over the map
- an alternate way is to use the tiles, but the Jim procedure should be used as default

In [ ]:
#grab the data and geometries, note the extra steps required for the pgd geometry fetches
queryset = (Queryset("m_fatalities_conflict_pgm", "priogrid_month")
            #log transformed
            .with_column(Column("ln_ged_sb", from_table = "ged2_pgm", from_column = "ged_sb_best_sum_nokgi")
                         .transform.ops.ln()
                         .transform.missing.fill()
                        )
                
            #.with_column(Column('name', from_table='country', from_column= 'name')
            #the above does not seem to want to work 
                         
            .with_theme("fatalities")
                         
            .describe("""Fatalities conflict history, pgm level
            Fatalities description 
                
            """)
            
           )

pg_master = queryset.publish().fetch()

#fetching geometries
engine = sa.create_engine(source_db_path)
gdf_pid = gpd.GeoDataFrame.from_postgis(
    "SELECT id as priogrid_gid, in_africa, in_me, geom FROM prod.priogrid", 
    engine, 
    geom_col='geom'
)
gdf_pid = gdf_pid.to_crs(4326)

#now grab the country names
pg_country_queryset = (Queryset("m_associate_country_2_priogrid", "priogrid_month")
              #CONFLICT TYPES NOT LOGGED and NOT time lagged 
            #.with_column(Column("month_id", from_table = "priogrid_month", from_column = "month_id"))
            .with_column(Column('name', from_table = 'country', from_column = 'name'))
                             
            .with_theme("Mapping")
                         
            .describe("""Mapping, allows for association of priogrid with country 
                
            """)
            
           )
pg_country = pg_country_queryset.publish().fetch().reset_index()
#pg_country_cleaned = pg_country.reset_index().drop(['month_id'], axis = 1).drop_duplicates(subset=['priogrid_gid'])

gdf_pid_master = pd.merge(left = gdf_pid, right = pg_country, left_on = 'priogrid_gid', 
                          right_on = 'priogrid_gid', how = 'right').set_index(['month_id', 'priogrid_gid'])


#grab country geometries for borders creation for Jim procedure
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)

In [ ]:
#pgm level demonstration

data = pg_master.copy()
gdf = gdf_pid_master.copy()
data_pid = pd.merge(left = data, right = gdf, left_index = True, right_index = True, how = 'left')
gdf = gpd.GeoDataFrame(data_pid, geometry="geom")
gdf_c = gdf_ci_master.copy()

#you can adjust these to see various effects and combinations
#please do not use the experimental labels as the name will be the name of priogrid_cell
background2 = 'StamenTerrainBackground' #use None to see the need for background use
textbox_label_font = 20
masking_location_choice = 'Somalia' #choices allowed are 'africa', 'ame', or country_id number


pgm_masked=Mapper2(
    width=20,
    height=20,
    frame_on=True,
    title='mask with layers',
    bbox=bbox_from_cid('ame')
).add_layer(
    gdf=gdf.loc[450],
    map_dictionary = dictionary_stand_1p,
    cmap = 'binary',
    transparency = 1.0,
    edgecolor="black",
    linewidth=0.5,
    column="ln_ged_sb"
).add_mask(
    gdf = gdf.loc[450],
    map_dictionary = dictionary_stand_1p,
    cmap = 'rainbow',
    transparency = 1,
    masking_location = 'Sudan',
    column='ln_ged_sb',
    background = None,
    edgecolor="black",
    linewidth=0.5
).add_views_textbox(
    text='hello world, hello world',
    textsize=textbox_label_font)

#Jim procedure for country border creation using the geometries inherent to views3
#note that for Jim procedure use plt.savefig to save the file
ax = pgm_masked.ax
gdf_c.plot(ax=ax,edgecolor='black',linewidth=2.0,facecolor='None')
plt.savefig(home+folder+'/pgm_masked', dpi = 300)